# Demand side flexibility


In [37]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [38]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [39]:
SCALE_MWH = 1000
SCALE_DOLLAR = 1000;

In [40]:
fixed_cost = CSV.read("fixed_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Fixed Cost,Fixed OM cost
,Float64,Float64
1,543.5,21.7531
2,666.821,28.7795
3,1110.5,37.5
4,3415.0,103.5
5,434.537,0.0
6,186.085,23.3723


In [41]:
variable_cost = CSV.read("variable_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Variable OM,Ramp,Start
,Float64,Float64,Float64
1,0.0,0.0,0.0
2,0.0,0.0,0.002
3,0.00589,0.006,0.035
4,0.011176,0.015,0.07
5,0.0,0.001,0.0
6,0.0,0.0,0.0


In [42]:
co2_intensity = CSV.read("co2_intensity.csv",DataFrame)

Row,CO2 Intensity
,Int64
1,0
2,0
3,452
4,965
5,0
6,0


In [43]:
P_max_s = 95897829.43 / SCALE_MWH
P_max_w = 11158008 / SCALE_MWH;
P_max_s, P_max_w

(95897.82943000001, 11158.008)

In [44]:
Availability_matrix = CSV.read("Availability.csv", DataFrame);
println(size(Availability_matrix))

(8760, 5)


In [45]:
demand = CSV.read("demand.csv", DataFrame) ./ SCALE_MWH;
#demand = CSV.read("demand_flex_vre50.csv", DataFrame) ./ SCALE_MWH;
println(size(demand))
first(demand, 5)

(8760, 4)


Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,177.72,109.564,169.055,55.3949
2,174.776,108.33,164.425,37.6361
3,173.069,102.035,165.18,33.1084
4,171.107,96.909,168.945,29.3248
5,171.561,94.8884,173.68,23.6573


In [46]:
ramp_limits = CSV.read("ramp_limits.csv", DataFrame)

Row,Ramp_limit
,Float64
1,1.0
2,0.1
3,0.025
4,0.03
5,1.0
6,1.0


In [47]:
curtailment_costs = [0, 2, 0, 0, 0, 0] ./ SCALE_DOLLAR

6-element Vector{Float64}:
 0.0
 0.002
 0.0
 0.0
 0.0
 0.0

Test without storage first

In [48]:
hrs = 4000
hre = 5000
sel_hrs = hrs:hre
println(sel_hrs)
techs = 5
n_eu = 4
# pad year by 1 day
add_h = 24

println("Cost")
C_FC = fixed_cost[1:techs,1]
C_FOM = fixed_cost[1:techs,2]
E_CO2 = co2_intensity[1:techs,1]
C_VOM = variable_cost[1:techs,1]
C_ramp = variable_cost[1:techs,2]
C_start = variable_cost[1:techs,3]
C_curt = curtailment_costs[1:techs]

println("Demand")
D_b = demand[sel_hrs, 1:4] # Baseline demand with no flex
println(size(D_b))
D_b = vcat(reverse(reverse(demand)[1:add_h, 1:n_eu]), D_b, D_b[1:24, 1:n_eu])
println(size(D_b))
total_demand = sum(eachcol(D_b))
X_D = total_demand

A_r = ramp_limits[1:techs, 1]
println("Availability")
A_s = Availability_matrix[sel_hrs,1:techs]
println("Try reverse")
A_s = vcat(reverse(reverse(Availability_matrix)[1:add_h, 1:techs]), A_s, A_s[1:24, 1:techs])
println(size(A_s))
# over time, opt each year with worst day for availability and highest demand?

4000:5000
Cost
Demand
(1001, 4)
(1049, 4)
Availability
Try reverse
(1049, 5)


In [49]:
println(size(X_D))

(1049,)


Find the worst days of the year

In [50]:
ls_max_ind = ones((1,10))
copy_D_b = copy(X_D)
println(copy_D_b)
println(findmax([1, 2, 3, 4])) 
println(findmax(copy_D_b)) 
a, b = findmax(copy_D_b)
println(b)
for i =1:10
    val, ind_max = findmax(copy_D_b)
    
    copy_D_b[Int(ind_max)] =0

    ls_max_ind[i] = ind_max
end

println(ls_max_ind)

[601.98634771, 572.19208745, 565.1589935700001, 564.2727646599999, 567.7776240100001, 587.91373415, 625.17438904, 652.71380841, 663.18950937, 677.4663077299999, 682.52435135, 695.21497165, 707.0570819899999, 702.69476106, 701.3263457, 714.40360029, 761.54746145, 812.1953581800001, 852.11807554, 847.01161626, 809.6992149299999, 744.8076524, 687.6719301, 597.5594579000001, 921.3793809, 967.4563828, 981.6986582000001, 900.4736553, 796.7927491, 745.4476346, 694.34588959, 627.5673007099999, 571.70912728, 544.0427434, 502.71945153, 484.04882013, 472.75419975999995, 466.76251538, 474.37910623999994, 501.33120693, 547.95260064, 608.7178000000001, 676.4602342000001, 734.3961337999999, 786.2933908999998, 833.6811581000001, 852.2652538, 867.4257135, 882.8193602, 894.2650286, 869.1695817, 864.5601316, 840.7826511, 801.7207489, 746.4254928, 674.5307611000001, 601.4775906, 537.70460823, 492.97483293, 473.07996698, 460.92197790999995, 451.92042458000003, 454.50073467, 477.63886734000005, 522.07671973

# Constants

In [51]:
eta = 0.8; # Efficiency of the battery
CO2_price = 100; # price of the CO2 per ton
H = 2;

# Model

In [52]:
model = Model(Gurobi.Optimizer) # TODO: options? other optimizers?
set_optimizer_attribute(model, "NonConvex", 2)
# set_optimizer_attribute(model, "IterationLimit", 10000)
println("Model: initialized!\n")

n = size(C_FC, 1) # number of technologies
println("Number of technologies: ", n)
#T = size(D_b, 1) # hours in simulation
T = 50
T_start = 987
time_ls_opt = T_start-20:T_start+20

println("Timespan: ", T)
#time_ls = 1+add_h:T-add_h
time_ls = T_start-20+add_h:T_start+20-add_h
vre_ls = 1:2
fossil_ls = 3:4
batt_ind = 5
# Non-negativity constraints
# The installed power capacity of the mix
@variable(model, P[1:n] >= 0)

# The hourly power generation of each technology
@variable(model, X_gen[time_ls_opt, 1:n] >= 0); # hourly production/use for each technology

# Curtailment (only renewables?)
@variable(model, X_cur[time_ls_opt, 1:n] >= 0); # hourly production/use for each technology

###### CONSTRAINTS ######

# Demand constraint - meet demand per hour
println("Demand constraint")
@constraint(model, Con_dh[t = time_ls_opt], sum(X_gen[t, i] for i in 1:n) >= X_D[t]); # TODO: add curtailment
# Meet total demand
# @constraint(model, Con_d, sum(X_gen[t, i] for i in 1:n for t in time_ls_opt) >= sum(total_demand));

# Installed capacity renewables < max capacity
println("Renewable maximum installed capacity constraint")
@constraint(model, Con_sgen, P[1] <= P_max_s);
@constraint(model, Con_wgen, P[2] <= P_max_w);

println("Renewable minimum installed capacity constraint")
#@constraint(model, Con_solar_lb, P[1] >= 50);
#@constraint(model, Con_wind_lb, P[2] >= 600);

# Storage variable
println("Storage variable")
@variable(model, 0<= X_soc[time_ls_opt] <=1)
@variable(model, X_ch[time_ls_opt]>=0)

# Cannot produce more than the installed power
# println("Installed power constraint")
# @constraint(model, Con_p[i = 1:n, t = time_ls_opt], X_gen[t, i] <= P[i])

# For each tech, at each hour, usage cannot exceed available capacity
println("Availability & installed power constraint")
@constraint(model, [i = 1:n, t = time_ls_opt], X_gen[t, i] <= P[i]*A_s[t, i])
# Renewables produce maximum possible capacity
@constraint(model, [i = vre_ls, t = time_ls_opt], X_gen[t, i] >= 0.7*P[i]*A_s[t, i]) 


println("Availability & installed power constraint")
@constraint(model, [i = 1:n, t = time_ls_opt], X_gen[t, i] <= P[i]*A_s[t, i])

println("Curtailment definition")
@constraint(model, [t = time_ls_opt], sum(X_cur[t, i] for i in 1:n) == sum(X_gen[t, i] for i in 1:n) - X_D[t]) 
@constraint(model, [i = 1:n, t = time_ls_opt], X_cur[t, i] <= X_gen[t, i])

# Cannot produce less than the minimum stable generation: X_gen must be zero or min to max
# min_usage = [0, 0.1, 0, 0] #TODO - cant be more than ramp
# @constraint(model, [i = 1:n, t = time_ls_opt], X_gen[t, i] >= P[i]*min_usage[i])

# For each tech, at each hour, change in production cannot exceed ramp rate (TODO might need to convert this to time)
println("Ramp constraint")
# Dummy rate of change variable
# @variable(model, X_roc[time_ls, 1:n])
# @constraint(model, [i = 1:4, t = time_ls], X_roc[t, i] == X_gen[t, i]-X_gen[limit_bound_max(t+1, T), i])

# Auxiliary variables to linearize the rate of change constraint
@constraint(model, Con_rdwn[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) >= -A_r[i]*X_gen[t, i]) 
@constraint(model, Con_rup[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) <= A_r[i]*X_gen[t, i]);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Set parameter NonConvex to value 2
Model: initialized!

Number of technologies: 5
Timespan: 50


Demand constraint
Renewable maximum installed capacity constraint
Renewable minimum installed capacity constraint
Storage variable
Availability & installed power constraint
Availability & installed power constraint
Curtailment definition
Ramp constraint


## Objective

In [53]:
# # Add objective function: minimize the total costs TODO: discount rates
# function start_cost(x_0, x_1, i)
#     if x_0 == 0
#         C_start[i]
#     else
#         0
#     end
# end

#TODO RAMP AND STARTUP COSTS

@objective(model, Min, 
    (
        sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:n) +
        sum(sum((C_VOM[i] + 100* E_CO2[i])* X_gen[t, i] for i in 1:n) for t in time_ls_opt) +
        sum(sum(C_curt[i] * X_cur[t, i] for i in 1:n) for t in time_ls)
));

# Storage model

Variable to indicate if $x_{g,t}<x_{d,t}$

In [54]:
M = 10000000


# Need to put this conditions above
@variable(model, Z_X_gen_greater_X_D[time_ls_opt], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in time_ls_opt], sum(X_gen[t,i] for i in 1:4) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in time_ls_opt], -sum(X_gen[t,i] for i in 1:4) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))



1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 967:1007
And data, a 41-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 Z_X_gen_greater_X_D_2[967] : -X_gen[967,1] - X_gen[967,2] - X_gen[967,3] - X_gen[967,4] + 10000000 Z_X_gen_greater_X_D[967] ≤ 9.99920694074012e6
 Z_X_gen_greater_X_D_2[968] : -X_gen[968,1] - X_gen[968,2] - X_gen[968,3] - X_gen[968,4] + 10000000 Z_X_gen_greater_X_D[968] ≤ 9.99928522307209e6
 Z_X_gen_greater_X_D_2[969] : -X_gen[969,1] - X_gen[969,2] - X_gen[969,3] - X_gen[969,4] + 10000000 Z_X_gen_greater_X_D[969] ≤ 9.99935046458875e6
 Z_X_gen_greater_X_D_2[970] : -X_gen[970,1] - X_gen[970,2] - X_gen[970,3] - X_gen[970,4] + 10000000 Z_X_gen_greater_X_D[970] ≤ 9.99940062580694e6
 

Create the variable X_min = min(X_1, X_2, X_3)

In [55]:
# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P3], (1- X_soc[t])*P[5]* H]
@variable(model, z1[t in time_ls_opt], Bin)
@variable(model, z2[t in time_ls_opt], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in time_ls_opt], eta*P[5] - eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in time_ls_opt], -eta*P[5] + eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[time_ls_opt])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in time_ls_opt], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in time_ls_opt], min_X_1_X_2[t] <= eta*P[5])
@constraint(model, X_1_X_2_higher_1[t in time_ls_opt], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in time_ls_opt], min_X_1_X_2[t] >= eta*P[5] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in time_ls_opt], (1- X_soc[t])* P[5] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in time_ls_opt], - (1- X_soc[t]) * P[5] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[time_ls_opt])

@constraint(model, X_1_X_2_X_3_lower_1[t in time_ls_opt], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in time_ls_opt], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[3]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in time_ls_opt], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in time_ls_opt], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[3]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of $X_{CH}$

In [56]:
#Expression of X_ch
@constraint(model, X_ch_expression[t in time_ls_opt], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P3]* H]

In [57]:
@variable(model, y2[time_ls_opt], Bin)

# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, y_2_1[t in time_ls_opt], X_soc[t]* P[5] * H - P[5] <=  M*y2[t])
@constraint(model, y_2_2[t in time_ls_opt], - X_soc[t] * P[5] * H + P[5] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_extra_prod_P5_soc[time_ls_opt])

@constraint(model, min_extra_prod_P5_soc_lower_1[t in time_ls_opt], min_extra_prod_P5_soc[t] <= P[5])
@constraint(model, min_extra_prod_P5_soc_lower_2[t in time_ls_opt], min_extra_prod_P5_soc[t] <= X_soc[t]*P[5]* H)
@constraint(model, min_extra_prod_P5_soc_higher_1[t in time_ls_opt], min_extra_prod_P5_soc[t] >= P[5] - M*(1-y2[t]))
@constraint(model, min_extra_prod_P5_soc_higher_2[t in time_ls_opt], min_extra_prod_P5_soc[t] >= X_soc[t]*P[5]* H - M*y2[t]);



 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of $X_{dch}$

In [58]:
@variable(model, batt_dispatch[time_ls_opt], Bin)

#Expression of X_dch
# issue with this constraint
@constraint(model, X_dch_expression[t in time_ls_opt], X_gen[t,5]  <= batt_dispatch[t]*min_extra_prod_P5_soc[t]);



Expression of state of charge

In [59]:
# First hour need to be equal to one
# Start with a charged battery
@constraint(model, SOC_t_1, X_soc[time_ls_opt[1]] == 1)

#
@constraint(model, SOC_t[t in time_ls_opt[2]:time_ls_opt[end]], (X_soc[t]-X_soc[t-1])*H*P[5] == (X_ch[t-1]- X_gen[t-1,5]));


In [60]:
#Optimize
optimize!(model)

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1274 rows, 825 columns and 3944 nonzeros
Model fingerprint: 0x28d94a95
Model has 450 quadratic constraints
Variable types: 620 continuous, 205 integer (205 binary)
Coefficient statistics:
  Matrix range     [8e-04, 1e+07]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+07]
  Objective range  [4e+02, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
  QRHS range       [1e+07, 1e+07]
Presolve added 0 rows and 51 columns
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2074 rows, 998 columns, 5953 nonzeros
Presolved model has 199 bilinear constraint(s)
Variable types: 793 continuous, 205 integer (205 binary)

Root relaxation: objective 4.920360e+05, 662 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    

## Results

In [61]:
println(objective_value(model))

2.8003864805364525e6


In [62]:
value.(P)

5-element Vector{Float64}:
  276.80287361233115
 2445.5710699231063
  140.96037591514104
    0.0
 1797.220626625918

In [63]:
X_gen_matrix = value.(X_gen);
    X_gen_matrix = transpose(X_gen_matrix)
size(X_gen_matrix)

(5, 41)

In [64]:
X_cur_matrix = value.(X_cur);
    X_cur_matrix = transpose(X_cur_matrix)
size(X_cur_matrix)

(5, 41)

In [65]:
println(X_D[100-24:100+24])

[851.7888621999999, 833.7761584, 797.8861195000001, 742.8704905, 669.0114247, 595.0740793, 515.5855982400001, 486.33434914000003, 472.16013506, 474.08693584, 488.06519483, 524.1631217400001, 583.00495065, 655.8571122999999, 706.4104289, 743.8290663999999, 787.2777206000001, 832.4878927000001, 867.6460957000002, 888.2089695, 914.9671275000001, 961.1138447999999, 1008.7749896, 1027.2238968, 947.2244089999999, 840.7463304, 789.9000179000001, 730.01119439, 654.7683562, 593.85769096, 546.75331423, 517.86603836, 499.5879964100001, 498.11451511999996, 509.32624235, 545.19362947, 602.9232835700001, 673.5155715999999, 723.0310515, 758.6260843, 800.8154106, 846.2188444, 881.4700204, 902.4501260999999, 930.3252605, 976.9818934, 1025.2017402000001, 1044.8704355, 968.3330818000001]


In [71]:
using Plots
println(T)
@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[1, time_ls]
b = X_gen_matrix[2, time_ls]
c = X_gen_matrix[3, time_ls]
d = X_gen_matrix[4, time_ls]
e = X_gen_matrix[5, time_ls]
a_c = X_cur_matrix[1, time_ls] * -1
b_c = X_cur_matrix[2, time_ls] * -1
c_c = X_cur_matrix[3, time_ls] * -1
d_c = X_cur_matrix[4, time_ls] * -1
x = time_ls


time_ls = time_ls_opt
plotly()
stackedarea(x, [d c b a e], labels=["Coal" "CCGT" "Wind" "Solar" "Battery"], lw=0)
stackedarea!(x, [d_c c_c b_c a_c], labels=["Coal, curtail" "CCGT, curtial" "Wind, curtail" "Solar, curtail"], lw=0)
plot!(x, total_demand[time_ls], label="Demand", lc=:black, lw=1, ls=:dot)
# plot!(x, value.(X_D)[:], label="Demand, shifted", lc=:black, lw=2)
plot!(size=(800,400))

50


In [72]:
# Compare availability of solar to wind
plotly()
plot(x, value.(A_s)[time_ls, 1]*P_max_s, label="Solar", lc=:black, lw=1)
plot!(x, a, label="Solar Prod", lc=:blue, lw=1)
plot!(x, value.(A_s)[time_ls, 2]*P_max_w, label="Wind", lc=:black, lw=1, ls=:dash)
plot!(size=(1000,400))

In [73]:
plotly()
ls_X_SOC = []
for elem in value.(X_soc)
    #print(elem)
    ls_X_SOC = [ls_X_SOC; elem]
end
println(ls_X_SOC)


Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9695220479101945, 0.9041883847136398, 0.8205088657179023, 0.734397915142612, 0.6327673300316842, 0.5263926112517416, 0.4159377797848854, 0.31198667854472006, 0.223084151098696, 0.141628536636609, 0.06625362578378735, 0.0, 0.0, 0.05545855316349717, 0.12954123145126434, 0.19781342547876613, 0.26073086562855063, 0.3187135382446686, 0.3721484891212304, 0.4213924071114315, 0.4667740050961826, 0.5085962142075598, 0.5471382059537054, 0.5826572557438493, 0.5759508806801569, 0.5141354097812307, 0.43376123257164967, 0.3420033823534558, 0.24339909085783315, 0.14330526493385623, 0.05760360239079269]


In [69]:
plot(time_ls_opt, ls_X_SOC, label="Battery state of charge", lc=:red, lw=2, ls=:dot)
plot!(size=(800,400))

In [70]:
plot(time_ls_opt, value.(X_gen[:,5]), label="X min", lc=:red, lw=2, ls=:dot)
plot!(size=(800,400))

LoadError: DenseAxisArray does not support this operation.